In [22]:
from typing import List, Dict, Union
from attr import dataclass
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy

In [9]:
analyzer = SentimentIntensityAnalyzer()
nlp = spacy.load('en')

In [16]:
test_sentence = 'Sen. Tammy Duckworth criticizes Trump\'s response to Iranian missile strikes'

In [17]:
doc = nlp(test_sentence)
subtokens = [token for token in doc] 

In [18]:
print([toc.dep_ + ":" + toc.text for toc in subtokens])

['compound:Sen.', 'compound:Tammy', 'nsubj:Duckworth', 'compound:criticizes', 'poss:Trump', "case:'s", 'ROOT:response', 'prep:to', 'amod:Iranian', 'compound:missile', 'pobj:strikes']


In [19]:
print(subtokens)

[Sen., Tammy, Duckworth, criticizes, Trump, 's, response, to, Iranian, missile, strikes]


In [23]:
@dataclass(init=False)
class AnalysisResult:
    sentiment: float
    subjectResults: Dict[str, float]

class SentimentAnalyzer:

    def __init__(self, subjects: List[str] = None):
        self._analyzer = SentimentIntensityAnalyzer()
        self._nlp = spacy.load('en')
        self._subjects = subjects

    def analyze(self, sentence: str) -> AnalysisResult:
        analysis_result = AnalysisResult()
        scores = self._analyzer.polarity_scores(sentence)
        analysis_result.sentiment = self._normalize_score(scores['compound'])
        analysis_result.subjectResults = {}

        token_scores = []
        
        for token in doc:
            subject = self._lookup_subject(token.text)
            if subject is None:
                continue
            scores = self._analyzer.polarity_scores(token.sent.text)
            score = scores['compound']
            adjusted_score = self._normalize_score(score)
            analysis_result.subjectResults[subject] = adjusted_score

        return analysis_result

    @staticmethod
    def _normalize_score(score: float):
        return score * 5 + 5

    def _lookup_subject(self, sentence_subject: str) -> Union[str, None]:
        if self._subjects is None:
            return None

        for subject in self._subjects:
            subject_words = subject.split()
            for subject_word in subject_words:
                if sentence_subject.lower() == subject_word.lower():
                    return subject
        return None
    

In [24]:
analyzer = SentimentAnalyzer(['Donald Trump', 'Tammy Duckworth'])

In [25]:
analyzer.analyze(test_sentence)

AnalysisResult(sentiment=2.0029999999999997, subjectResults={'Tammy Duckworth': 2.0029999999999997})